# N starvation 

## Which N sources allow HOT1A3 to grow

check out Escher visualization https://escher.readthedocs.io/en/latest/getting_started.html

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize


In [2]:
import subprocess
import time


In [3]:
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import production_envelope


In [4]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [5]:
sns.hls_palette

<function seaborn.palettes.hls_palette(n_colors=6, h=0.01, l=0.6, s=0.65, as_cmap=False)>

In [6]:
def get_elements(rid):
    r = model.reactions.get_by_id(rid)
    m = r.reactants[0]    
    elements = m.elements.copy()
    
    res = {
        'reaction' : rid,
        'metabolite' : m.name,
        'mid' : m.id,
        'N' : 'N' in elements,
        'C' : 'C' in elements,
        
    }
    return pd.Series(res)

# Import model and manipulate based on Ofaim at el

In [7]:
import os
model_dpath = os.path.join('..', 'Models')
model_fname = 'HOT1A3_ncbi.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [8]:
model = cobra.io.read_sbml_model(model_fpath)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [9]:
exchange_ids = [r.id for r in model.exchanges] # + ["R00024"]

In [10]:
print('reactions', len(model.reactions))
print('metabolites', len(model.metabolites))
print('genes', len(model.genes))

reactions 2448
metabolites 1615
genes 1025


In [11]:
model

Name,HOT1A3_ncbi
Memory address,0x07f695c34d640
Number of metabolites,1615
Number of reactions,2448
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


In [12]:
cobra.Configuration()

Attribute,Description,Value
solver,Mathematical optimization solver,gurobi
tolerance,"General solver tolerance (feasibility, integrality, etc.)",1e-07
lower_bound,Default reaction lower bound,-1000.0
upper_bound,Default reaction upper bound,1000.0
processes,Number of parallel processes,11
cache_directory,Path for the model cache,/home/oweissberg/.cache/cobrapy
max_cache_size,Maximum cache size in bytes,104857600
cache_expiration,Model cache expiration time in seconds (if any),None


In [13]:
exchange_ids

['EX_14glucan_e',
 'EX_25dkglcn_e',
 'EX_2ameph_e',
 'EX_2m35mdntha_e',
 'EX_2pglyc_e',
 'EX_35dnta_e',
 'EX_3h4atb_e',
 'EX_3mb_e',
 'EX_4hba_e',
 'EX_4hbald_e',
 'EX_4hbz_e',
 'EX_4hphac_e',
 'EX_5dglcn_e',
 'EX_5drib_e',
 'EX_5mdru1p_e',
 'EX_6atha_e',
 'EX_LalaLglu_e',
 'EX_abg4_e',
 'EX_abt_e',
 'EX_ac_e',
 'EX_acac_e',
 'EX_acald_e',
 'EX_acgam_e',
 'EX_acmana_e',
 'EX_actn__R_e',
 'EX_ade_e',
 'EX_adn_e',
 'EX_ala_L_asp__L_e',
 'EX_ala_L_glu__L_e',
 'EX_ala_L_thr__L_e',
 'EX_ala__D_e',
 'EX_ala__L_e',
 'EX_ala_gln_e',
 'EX_ala_his_e',
 'EX_ala_leu_e',
 'EX_alagly_e',
 'EX_alahis_e',
 'EX_alaleu_e',
 'EX_alathr_e',
 'EX_alatrp_e',
 'EX_alltn_e',
 'EX_arab__L_e',
 'EX_arbt6p_e',
 'EX_arbt_e',
 'EX_arg__L_e',
 'EX_ascb__L_e',
 'EX_asn__L_e',
 'EX_aso3_e',
 'EX_aso4_e',
 'EX_asp__L_e',
 'EX_bhb_e',
 'EX_btn_e',
 'EX_but_e',
 'EX_bz_e',
 'EX_ca2_e',
 'EX_cellb_e',
 'EX_cgly_e',
 'EX_chol_e',
 'EX_chols_e',
 'EX_chtbs_e',
 'EX_cinnm_e',
 'EX_cit_e',
 'EX_cl_e',
 'EX_co2_e',
 'EX_co_e'

In [14]:
model.reactions.get_by_id('EX_cyan_e')

Reaction identifier,EX_cyan_e
Name,R_EX_cyan_e
Memory address,0x7f695a660040
Stoichiometry,cyan_e <=> Hydrogen cyanide <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
model.reactions.get_by_id('Growth')

Reaction identifier,Growth
Name,Biomass reaction
Memory address,0x7f695a5f7fd0
Stoichiometry,0.000223 10fthf_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 0.0001... 0.000223 10-Formyltetrahydrofolate + 0.513689 L-Alanine + 0.000223 S-Adenosyl-L-methionine + 0.295792 L-Arginine + 0.241055 L-Asparagine + 0.241055 L-Aspartate + 54.124831 ATP C10H12N5O13P3 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


# Identify N sources where MED4 grows on

In [16]:
len(model.medium)

257

In [17]:
len(exchange_ids)

257

In [18]:
# copied from MED4 model
pro99_mediums = ['Ammonia', 'Cadmium', 'Ca2+', 'Cl-', 'Cobalt ion', 'Copper',
       'Orthophosphate', 'H2O', 'HCO3-', 'H+', 'Fe2+', 'Photon',
       'Magnesium cation', 'Molybdenum', 'O2', 'K+', 'Selenate', 'Na+',
       'Strontium cation', 'Sulfate', 'Zn2+', 'Hydrogen peroxide']
print('\n'.join(pro99_mediums))

Ammonia
Cadmium
Ca2+
Cl-
Cobalt ion
Copper
Orthophosphate
H2O
HCO3-
H+
Fe2+
Photon
Magnesium cation
Molybdenum
O2
K+
Selenate
Na+
Strontium cation
Sulfate
Zn2+
Hydrogen peroxide


In [19]:
# what are the metabolites in the medium?
df_medium = pd.DataFrame({'mediumExchange' : model.medium.keys()})
df_medium = pd.DataFrame.from_records(df_medium.mediumExchange.apply(get_elements))


In [20]:
df_medium

,reaction,metabolite,mid,N,C
0,EX_14glucan_e,"1,4-alpha-D-glucan",14glucan_e,False,True
1,EX_25dkglcn_e,"2,5-diketo-D-gluconate",25dkglcn_e,False,True
2,EX_2ameph_e,2-Aminoethylphosphonate,2ameph_e,True,True
3,EX_2m35mdntha_e,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,2m35mdntha_e,True,False
4,EX_2pglyc_e,2-Phosphoglycolate,2pglyc_e,False,True
...,...,...,...,...,...
252,EX_vanlt_e,Vanillate,vanlt_e,False,True
253,EX_xan_e,Xanthine,xan_e,True,True
254,EX_xtsn_e,Xanthosine,xtsn_e,True,True
255,EX_xyl__D_e,D-Xylose,xyl__D_e,False,True


In [21]:
pro99_df = pd.read_excel('HOT1A3_medium.xlsx')

In [22]:
pro99_df.head()

,reaction,metabolite,N,C,pro99,MED4 metabolite,promm
0,EX_actn__R_e,R Acetoin C4H8O2,False,True,NaN,NaN,NaN
1,EX_bhb_e,(R)-3-Hydroxybutanoate,False,True,NaN,NaN,NaN
2,EX_12ppd__R_e,"(R)-Propane-1,2-diol",False,True,NaN,NaN,NaN
3,EX_12ppd__S_e,"(S)-Propane-1,2-diol",False,True,NaN,NaN,NaN
4,EX_phedca_e,10 Phenyldecanoic acid,False,True,NaN,NaN,NaN


In [23]:
df_medium.loc[df_medium.metabolite.str.contains('ic')]

,reaction,metabolite,mid,N,C
6,EX_3h4atb_e,3 Hydroxy 4 acetylthiobutanoic acid,3h4atb_e,False,False
7,EX_3mb_e,3-Methylbutanoic acid,3mb_e,False,True
15,EX_6atha_e,6 acetylthiohexanoic acid,6atha_e,False,True
88,EX_fe3dcit_e,Fe(III)dicitrate,fe3dcit_e,False,True
89,EX_fe3pyovd_kt_e,Ferrypyoverdine P putida KT2440 specific,fe3pyovd_kt_e,True,True
91,EX_feoxam_e,Generic ferrioxamine-Fe-III,feoxam_e,True,True
141,EX_hco3_e,Bicarbonate,hco3_e,False,True
165,EX_lnlc_e,Linoleic acid (all cis C18:2) n-6,lnlc_e,False,True
198,EX_phedca_e,10 Phenyldecanoic acid,phedca_e,False,True
199,EX_phehpa_e,7 Phenylheptanoic acid,phehpa_e,False,True


In [24]:
pro99_df.loc[~pro99_df.reaction.isin(exchange_ids)]

,reaction,metabolite,N,C,pro99,MED4 metabolite,promm
2,EX_12ppd__R_e,"(R)-Propane-1,2-diol",False,True,NaN,NaN,NaN
3,EX_12ppd__S_e,"(S)-Propane-1,2-diol",False,True,NaN,NaN,NaN
12,EX_4abut_e,4-Aminobutanoate,True,True,NaN,NaN,NaN
42,EX_btoh_e,Butanol,False,True,NaN,NaN,NaN
47,EX_chor_e,Chorismate,False,True,NaN,NaN,NaN
53,EX_alaala_e,D-Alanyl-D-alanine,True,True,NaN,NaN,NaN
70,EX_glcn_e,D-Gluconate,False,True,NaN,NaN,NaN
74,EX_glyald_e,D-Glyceraldehyde,False,True,NaN,NaN,NaN
82,EX_ser__D_e,D-Serine,True,True,NaN,NaN,NaN
115,EX_gsn_e,Guanosine,True,True,NaN,NaN,NaN


In [25]:
pro99_df.loc[~pro99_df.reaction.isin(exchange_ids), 'promm'] = np.NaN

In [26]:
promm_exchanges = pro99_df.loc[pro99_df.pro99.isin(['yes']) | pro99_df.promm.isin(['yes']), 'reaction']
pro99_exchanges = pro99_df.loc[pro99_df.pro99.isin(['yes']), 'reaction']
other_exchahges = [m for m in exchange_ids if m not in promm_exchanges]

In [27]:
#[m for m in exchange_ids if m not in pro99_df.reaction.values]
df_medium.loc[~df_medium.reaction.isin(pro99_df.reaction), ['reaction', 'metabolite']]

,reaction,metabolite
0,EX_14glucan_e,"1,4-alpha-D-glucan"
4,EX_2pglyc_e,2-Phosphoglycolate
11,EX_4hphac_e,4-Hydroxyphenylacetate
14,EX_5mdru1p_e,5-Methylthio-5-deoxy-D-ribulose 1-phosphate
45,EX_ascb__L_e,L-Ascorbate
58,EX_chols_e,Choline sulfate
59,EX_chtbs_e,"N,N'-diacetylchitobiose"
70,EX_cynt_e,Cyanate
91,EX_feoxam_e,Generic ferrioxamine-Fe-III
92,EX_feoxam_un_e,Ferroxamine minus Fe(3)


In [28]:
#[m for m in exchange_ids if m not in pro99_df.reaction.values]
pro99_df.loc[~pro99_df.reaction.isin(df_medium.reaction), ['reaction', 'metabolite']]

,reaction,metabolite
2,EX_12ppd__R_e,"(R)-Propane-1,2-diol"
3,EX_12ppd__S_e,"(S)-Propane-1,2-diol"
12,EX_4abut_e,4-Aminobutanoate
42,EX_btoh_e,Butanol
47,EX_chor_e,Chorismate
53,EX_alaala_e,D-Alanyl-D-alanine
70,EX_glcn_e,D-Gluconate
74,EX_glyald_e,D-Glyceraldehyde
82,EX_ser__D_e,D-Serine
115,EX_gsn_e,Guanosine


In [29]:
pro99_df.head()

,reaction,metabolite,N,C,pro99,MED4 metabolite,promm
0,EX_actn__R_e,R Acetoin C4H8O2,False,True,NaN,NaN,NaN
1,EX_bhb_e,(R)-3-Hydroxybutanoate,False,True,NaN,NaN,NaN
2,EX_12ppd__R_e,"(R)-Propane-1,2-diol",False,True,NaN,NaN,NaN
3,EX_12ppd__S_e,"(S)-Propane-1,2-diol",False,True,NaN,NaN,NaN
4,EX_phedca_e,10 Phenyldecanoic acid,False,True,NaN,NaN,NaN


In [30]:
pro99_df.columns

Index(['reaction', 'metabolite', 'N', 'C', 'pro99', 'MED4 metabolite',
       'promm'],
      dtype='object')

In [31]:
df_medium.head()

,reaction,metabolite,mid,N,C
0,EX_14glucan_e,"1,4-alpha-D-glucan",14glucan_e,False,True
1,EX_25dkglcn_e,"2,5-diketo-D-gluconate",25dkglcn_e,False,True
2,EX_2ameph_e,2-Aminoethylphosphonate,2ameph_e,True,True
3,EX_2m35mdntha_e,N 2 methyl 3 5 dinitrophenyl 4 methyl 3 5 di...,2m35mdntha_e,True,False
4,EX_2pglyc_e,2-Phosphoglycolate,2pglyc_e,False,True


In [32]:
pd.merge(df_medium, pro99_df[['reaction', 'metabolite', 'pro99', 'MED4 metabolite',
       'promm']], on='reaction', how='left', suffixes=['', '_luca']).to_excel('HOT1A3_medium_NCBI.xlsx')

In [33]:
# does it grow on promm?
with model:
    medium = model.medium
    for m in medium:
        medium[m] = 0.0
    for m in promm_exchanges:
        medium[m] = 1000.0
    model.medium = medium
    print ('model medium', model.medium)
    solution = model.optimize()
    print(model.summary())


model medium {'EX_ac_e': 1000.0, 'EX_btn_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_glyc_e': 1000.0, 'EX_h2o2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_inost_e': 1000.0, 'EX_k_e': 1000.0, 'EX_lac__D_e': 1000.0, 'EX_lac__L_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_pyr_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_zn2_e': 1000.0}
Objective
1.0 Growth = 21.32652692214085

Uptake
------
Metabolite     Reaction     Flux  C-Number C-Flux
     ca2_e     EX_ca2_e    0.111         0  0.00%
      cl_e      EX_cl_e    0.111         0  0.00%
 cobalt2_e EX_cobalt2_e 0.002133         0  0.00%
     cu2_e     EX_cu2_e  0.01512         0  0.00%
     fe2_e     EX_fe2_e   0.3097         0  0.00%
    glyc_e    EX_glyc_e    774.8         3 61.18%
       h_e       EX_h_e    166.1         0  0.00%
   

In [34]:
# does it grow on pro99?
with model:
    medium = model.medium
    for m in medium:
        medium[m] = 0.0
    for m in pro99_exchanges:
        medium[m] = 1000.0
    model.medium = medium
    print ('model medium', model.medium)
    solution = model.optimize()
    print(model.summary())


model medium {'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_co2_e': 1000.0, 'EX_cobalt2_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_h2o2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_k_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0, 'EX_sel_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_zn2_e': 1000.0}
Objective
1.0 Growth = 0.0

Uptake
------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []

Secretion
---------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []



In [35]:
# try to add each exchange one by one
results = list()

def _check_medium(met1, met2=None):
    with model:
        medium = model.medium
        for m in medium:
            medium[m] = 0.0
        for m in pro99_exchanges:
            medium[m] = 1000.0
        medium["EX_nh4_e"] = 0.0
        medium[met1] = 1000.0
        model.medium = medium
        solution = model.optimize()
        return {'uptake' : met1, 'objective_value' : solution.objective_value}
results = [_check_medium(met) for met in exchange_ids]        

In [36]:
df = pd.DataFrame(results)

In [37]:
df_growing = df.loc[(df.objective_value > 1e-5), 
                    ['uptake','objective_value']
                   ].nlargest(n=1000, columns='objective_value').reset_index(drop=True)

In [38]:
tdf = pd.DataFrame.from_records(df_growing.uptake.apply(get_elements))
df_growing = pd.merge(df_growing, tdf, left_on='uptake', right_on='reaction')

In [39]:
df_growing

,uptake,objective_value,reaction,metabolite,mid,N,C
0,EX_cytd_e,30.106063,EX_cytd_e,Cytidine,cytd_e,True,True
1,EX_uri_e,25.045045,EX_uri_e,Uridine,uri_e,True,True
2,EX_adn_e,20.361252,EX_adn_e,Adenosine,adn_e,True,True
3,EX_ins_e,20.174939,EX_ins_e,Inosine,ins_e,True,True
4,EX_xtsn_e,19.652775,EX_xtsn_e,Xanthosine,xtsn_e,True,True
...,...,...,...,...,...,...,...
62,EX_ura_e,8.089889,EX_ura_e,Uracil,ura_e,True,True
63,EX_xan_e,7.895409,EX_xan_e,Xanthine,xan_e,True,True
64,EX_ptrc_e,7.342343,EX_ptrc_e,Putrescine,ptrc_e,True,True
65,EX_alltn_e,6.981746,EX_alltn_e,Allantoin,alltn_e,True,True


In [40]:
# check that the media is correct

In [ ]:
# try to add each pair of exchanges
results = list()
not_open_boundary_ids = [i for i in exchange_ids if (i not in pro99_exchanges) and (i not in df_growing.reaction.values)]

def _check_medium(met1, met2=None):
    with model:
        medium = model.medium
        for m in medium:
            medium[m] = 0.0
        for m in pro99_exchanges:
            medium[m] = 1000.0
        medium["EX_nh4_e"] = 0.0
        medium[met1] = 1000.0
        if met2 is not None:
            medium[met2] = 1000.0
        model.medium = medium
        solution = model.optimize()
        return {'uptake' : met1, 'uptake2' : met2, 'objective_value' : solution.objective_value}
results = [_check_medium(met1, met2) for met1, met2 in itertools.combinations(not_open_boundary_ids, 2)]        

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head()

In [ ]:
df_growing2 = df.loc[(df.objective_value > 1e-5), 
                    ['uptake','uptake2', 'objective_value']
                   ].nlargest(n=1000, columns='objective_value').reset_index(drop=True)

In [ ]:
df_growing2

In [ ]:
len(set(df_growing2.uptake.unique()) | set(df_growing2.uptake2.unique()))

In [ ]:
(set(df_growing2.uptake.unique()) | set(df_growing2.uptake2.unique())) & set(df_growing.uptake.unique())

In [ ]:
df_growing2[['reaction', 'metabolite','mid','N','C']] = df_growing2.uptake.apply(get_elements)
df_growing2[['reaction2', 'metabolite2','mid2','N2','C2']] = df_growing2.uptake2.apply(get_elements)

#df_growing = pd.merge(df_growing, tdf, left_on='uptake', right_on='reaction')

In [ ]:
df_growing2

### N sources that HOT1A3 can grow on 

In [ ]:
df_growing

### when all other potential mediums are available, HOT1A3 can grow on a combination of metabolites

- there is a list of 77 metabolites that enable growth when added to pro99. there are additional 174 metabolites that when all are available, it grows. for now do not pursue this list

In [ ]:
# open all additional mediums (all those that do not facilitate growth by themselves)
not_open_boundary_ids = [i for i in exchange_ids if (i not in pro99_exchanges) and 
                         (i not in df_growing.reaction.values) and
                         (i not in df_growing2.reaction.values) and
                         (i not in df_growing2.reaction2.values)
                        ]
with model:
    medium = model.medium
    for m in medium:
        medium[m] = 0.0
    for m in pro99_exchanges:
        medium[m] = 1000.0
    medium["EX_nh4_e"] = 0.0
    for i in not_open_boundary_ids:
        medium[i] = 1000.0
    model.medium = medium
    solution = model.optimize()
    print(model.summary())


In [ ]:
len(not_open_boundary_ids), len(exchange_ids), len(pro99_exchanges), len(df_growing.reaction)

In [ ]:
len(set(df_growing2.reaction) |set(df_growing2.reaction2))

In [ ]:
251-19-77

### Force HOT1A3 to uptake the potential N/C sources, limit growth (biomass), and see what is secreted

In [ ]:
def _force_feeding(met1):
    with model:
        medium = model.medium
        for m in medium:
            medium[m] = 0.0
        for m in pro99_exchanges:
            medium[m] = 1000.0
        medium["EX_nh4_e"] = 0.0
        medium[met1] = 1000.0
        model.medium = medium
        model.reactions.get_by_id(met1).upper_bound = -1e-5
        model.reactions.Growth.upper_bound = 1e-2
        solution = model.slim_optimize()
        #print(met1, model.summary())
        print(met1)
        fluxes = flux_variability_analysis(model, model.boundary, loopless=True, fraction_of_optimum=0.9, processes=11)
        fluxes['uptake'] = met1
        return fluxes
fluxes_df_list = [_force_feeding(met1) for met1 in df_growing['uptake']]

In [ ]:
def _force_feeding2(met1, met2):
    with model:
        medium = model.medium
        for m in medium:
            medium[m] = 0.0
        for m in pro99_exchanges:
            medium[m] = 1000.0
        medium["EX_nh4_e"] = 0.0
        medium[met1] = 1000.0
        medium[met2] = 1000.0
        model.medium = medium
        model.reactions.get_by_id(met1).upper_bound = -1e-5
        model.reactions.get_by_id(met2).upper_bound = -1e-5
        model.reactions.Growth.upper_bound = 1e-2
        solution = model.slim_optimize()
        #print(met1, model.summary())
        print(met1, met2)
        fluxes = flux_variability_analysis(model, model.boundary, loopless=True, fraction_of_optimum=0.9, processes=11)
        fluxes['uptake'] = met1
        fluxes['uptake2'] = met2
        return fluxes
fluxes2_df_list = [_force_feeding2(met1, met2) for met1, met2 in zip(df_growing2['uptake'], df_growing2['uptake2'])]

In [ ]:
force_feeding_df = pd.concat(fluxes_df_list)

In [ ]:
force_feeding2_df = pd.concat(fluxes2_df_list)

In [ ]:
force_feeding_df.index = force_feeding_df.index.set_names(['secretion'])
force_feeding_df = force_feeding_df.reset_index()
force_feeding_df.head()

In [ ]:
force_feeding2_df.index = force_feeding2_df.index.set_names(['secretion'])
force_feeding2_df = force_feeding2_df.reset_index()
force_feeding2_df.head()

In [ ]:
force_feeding_df.loc[force_feeding_df.maximum > 1e-3].groupby('secretion').minimum.count()

In [ ]:
force_feeding2_df.loc[force_feeding2_df.maximum > 1e-3].groupby('secretion').minimum.count()

In [ ]:
drop_columns = ['EX_co2_e',  'EX_fe3_e','EX_h2o_e',
       'EX_h_e', 'EX_o2_e',]

n_exchange_df = force_feeding_df.loc[(force_feeding_df.maximum > 1e-3) & (~force_feeding_df.secretion.isin(drop_columns))]
pdf = n_exchange_df.pivot_table(columns='uptake', index='secretion', values='maximum', fill_value=0)


In [ ]:
drop_columns = ['EX_co2_e',  'EX_fe3_e','EX_h2o_e',
       'EX_h_e', 'EX_o2_e',]

n_exchange2_df = force_feeding2_df.loc[(force_feeding2_df.maximum > 1e-3) & (~force_feeding2_df.secretion.isin(drop_columns))]
#pdf2 = n_exchange2_df.pivot_table(columns='uptake', index='secretion', values='maximum', fill_value=0)


In [ ]:
tdf = n_exchange_df.uptake.apply(get_elements)
n_exchange_df[['metabolite_uptake', 'mid_uptake', 'N_uptake', 'C_uptake']] = tdf[['metabolite', 'mid', 'N', 'C']]

tdf = n_exchange_df.secretion.apply(get_elements)
n_exchange_df[['metabolite_secretion', 'mid_secretion', 'N_secretion', 'C_secretion']] = tdf[['metabolite','mid', 'N', 'C']]

n_exchange_df.head()

In [ ]:
tdf = n_exchange2_df.uptake.apply(get_elements)
n_exchange2_df[['metabolite_uptake', 'mid_uptake', 'N_uptake', 'C_uptake']] = tdf[['metabolite', 'mid','N', 'C']]

tdf = n_exchange2_df.uptake2.apply(get_elements)
n_exchange2_df[['metabolite_uptake2', 'mid_uptake2', 'N_uptake2', 'C_uptake2']] = tdf[['metabolite', 'mid','N', 'C']]

tdf = n_exchange2_df.secretion.apply(get_elements)
n_exchange2_df[['metabolite_secretion', 'mid_secretion','N_secretion', 'C_secretion']] = tdf[['metabolite','mid', 'N', 'C']]

n_exchange2_df.head()

In [ ]:
col_anno = pd.DataFrame.from_records([get_elements(i) for i in pdf.columns])
row_anno = pd.DataFrame.from_records([get_elements(i) for i in pdf.index])
N_lut = {True: '#BF9270', False : '#FFEDDB'}
C_lut = {True: '#94B49F', False : '#FCF8E8'}
row_anno.N = row_anno.N.map(N_lut)
row_anno.C = row_anno.C.map(C_lut)
col_anno.N = col_anno.N.map(N_lut)
col_anno.C = col_anno.C.map(C_lut)
row_anno.index = row_anno.reaction
col_anno.index = col_anno.reaction

In [ ]:

sns.clustermap(pdf, norm=LogNorm(), 
               cmap='coolwarm', figsize=(25,15), 
               linewidths=0.1, #cbar_kws= dict(location='right'),
               row_colors=row_anno[['N', 'C']],
               col_colors=col_anno[['N', 'C']],
               yticklabels= row_anno.metabolite,
               xticklabels= col_anno.metabolite,
               colors_ratio=0.04,
              robust=True, cbar_pos=(1, .2, .03, .4),
              dendrogram_ratio=0.02)
plt.suptitle("Maximum FVA Flux - secretion when 'force fed'", y=1)

In [ ]:
pdf.shape

In [ ]:
len(exchange_ids)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('ALT_N_sources.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_growing.to_excel(writer, sheet_name='uptake')
df_growing2.to_excel(writer, sheet_name='uptake2')
n_exchange_df.to_excel(writer, sheet_name='secretion')
n_exchange2_df.to_excel(writer, sheet_name='secretion2')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
n_exchange_df.shape,n_exchange2_df.shape

In [ ]:
n_exchange_df.head(50)

In [ ]:
model.genes[30]